Prueba de jupyter notebook
Ejemplos de uso de networkx para futuras referencias

In [1]:
from google.colab import drive
import os, sys

drive.mount('/content/drive')
print(os.getcwd())
os.chdir('/content/drive/My Drive/2021-2022/2C/TFG')
sys.path.append('/content/drive/My Drive/2021-2022/2C/TFG/clustpy')

Mounted at /content/drive
/content


In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from random import randint
# For color mapping
import matplotlib.colors as colors
import matplotlib.cm as cmx
from networkx.algorithms import community

import mymod as mod
from mymod import mymodularity

import tabusearch
from tabusearch import tabu_modularity_optimization

import importlib
importlib.reload(mod)
importlib.reload(tabusearch)

<module 'tabusearch' from '/content/drive/My Drive/2021-2022/2C/TFG/clustpy/tabusearch.py'>

Obtenemos datos de los nodos y enlaces de los csv:

In [3]:
nodes_path = r'/content/drive/My Drive/2021-2022/2C/TFG/datos/Nodes_t1.csv'
edges_path = r'/content/drive/My Drive/2021-2022/2C/TFG/datos/Edges_t1.csv'
df_nodes = pd.read_csv(nodes_path, sep = ';',encoding='unicode_escape')
df_edges = pd.read_csv(edges_path, sep = ';',encoding='unicode_escape')

Creamos grafo de todos los alumnos:

In [4]:
MDG = nx.MultiDiGraph()
MDG.add_nodes_from(df_nodes['Nodes'])

tuples_from_to = []
for i in range(len(df_edges)):
    #tuples_from_to.append((df_edges['from'][i],df_edges['to'][i],{"weight":df_edges['weight'][i]}))
    tuples_from_to.append((df_edges['from'][i],df_edges['to'][i],df_edges['weight'][i]))

MDG.add_weighted_edges_from(tuples_from_to)
# Para obtener los todos los pesos:
#print([element[2]['weight'] for element in list(MDG.edges(data=True))])

Creamos grafos por aulas:

In [9]:
n_grados = df_nodes["Curso"].nunique()
grafos = []
results_dict = {}
for i in range(n_grados):
    j=i+1
    aulas = df_nodes[df_nodes["Curso"] == "%sº ESO" % j]['Grupo'].unique()
    n_aulas = df_nodes[df_nodes["Curso"] == "%sº ESO" % j]['Grupo'].nunique()

    for aula in aulas:
        newMDG = nx.MultiDiGraph()
        alumnos_en_aula = df_nodes[df_nodes["Curso"] == "%sº ESO" % j][df_nodes["Grupo"] == "%s" % aula]['Nodes']

        fromtos = []
        for alumno in alumnos_en_aula:
            fromtos.append(list(zip(list(df_edges.loc[df_edges['from'] == alumno]['from']),list(df_edges.loc[df_edges['from'] == alumno]['to']),list(df_edges.loc[df_edges['from'] == alumno]['weight']))))
        enlaces_aula = [item for sublist in fromtos for item in sublist]

        MDGaula = nx.MultiDiGraph()
        MDGaula.add_nodes_from(alumnos_en_aula)
        MDGaula.add_weighted_edges_from(enlaces_aula)
        grafos.append(MDGaula)
        results_dict["%sº ESO %s" % (j,aula)] = {'graph': MDGaula, 'init_modularity': None,'final_modularity': None}

for clase in list(results_dict.keys())[20:21]:#última clase (hay 21 clases)
    
    MDG_clase = results_dict[clase]['graph']
    pares = frozenset([nodo for nodo in MDG_clase.nodes if nodo%2 == 0])
    impares = frozenset([nodo for nodo in MDG_clase.nodes if nodo%2 != 0])
    c = [pares,impares]

    results_dict[clase]['init_modularity'] = mymodularity(MDG_clase,c[:])
    optimized_communities = tabu_modularity_optimization(MDG_clase,c[:])
    results_dict[clase]['final_modularity'] =  mymodularity(MDG_clase, optimized_communities[:])

print(results_dict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


maximo de vueltas inutiles 1
{'1º ESO A': {'graph': <networkx.classes.multidigraph.MultiDiGraph object at 0x7f42785a5190>, 'init_modularity': None, 'final_modularity': None}, '1º ESO B': {'graph': <networkx.classes.multidigraph.MultiDiGraph object at 0x7f4285709850>, 'init_modularity': None, 'final_modularity': None}, '1º ESO C': {'graph': <networkx.classes.multidigraph.MultiDiGraph object at 0x7f4277e71590>, 'init_modularity': None, 'final_modularity': None}, '1º ESO D': {'graph': <networkx.classes.multidigraph.MultiDiGraph object at 0x7f42785a5910>, 'init_modularity': None, 'final_modularity': None}, '1º ESO E': {'graph': <networkx.classes.multidigraph.MultiDiGraph object at 0x7f4278e02e50>, 'init_modularity': None, 'final_modularity': None}, '1º ESO F': {'graph': <networkx.classes.multidigraph.MultiDiGraph object at 0x7f4280e2c550>, 'init_modularity': None, 'final_modularity': None}, '1º ESO G': {'graph': <networkx.classes.multidigraph.MultiDiGraph object at 0x7f427b8c9610>, 'init_m

Dibujamos grafo de distintas formas
Primero coloreando los enlaces positivos y negativos

In [163]:
#Creamos leyenda en función de las relaciones en
# cNorm = colors.Normalize(vmin = df_edges['weight'].min(), vmax = df_edges['weight'].max())
# scalarMap = cmx.ScalarMappable(norm = cNorm, cmap = plt.cm.RdBu)
# ColorLegend = {'negativo fuerte': -2, 
#            'negativo débil': -1, 
#            'positivo fuerte': 2, 
#            'positivo débil': 1}

# f = plt.figure(1)
# ax = f.add_subplot(1,1,1)
# for label in ColorLegend:
#     ax.plot([0],[0],color=scalarMap.to_rgba(ColorLegend[label]),label=label)

# plt.legend(loc = 'lower right')
# nx.draw_shell(MDG,node_size=10,width = 0.1,vmin = df_edges['weight'].min(),vmax=df_edges['weight'].max(),edge_color=df_edges['weight'],edge_cmap=plt.cm.RdBu,ax=ax)
# plt.show()

In [164]:
# nx.draw_random(MDG,node_size=10,width = 0.1,edge_color=df_edges['weight'],edge_cmap=plt.cm.RdBu)

# plt.show()

In [165]:
# nx.draw_spring(MDG,node_size=10,width = 0.1,edge_color=df_edges['weight'],edge_cmap=plt.cm.RdBu)
# print(MDG)
# plt.show()

In [166]:
# c = list(community.modularity_max.greedy_modularity_communities(MDG1A))

pares = frozenset([alumnospares for alumnospares in MDG1A.nodes if alumnospares%2 == 0])
impares = frozenset([alumnosimpares for alumnosimpares in MDG1A.nodes if alumnosimpares%2 != 0])
print(pares)
c = [pares,impares]
# for k in c:
#     mdg_iter = nx.MultiDiGraph()
#     mdg_iter.add_nodes_from(k)
#     tuples_from_to = []
#     for i in k:
#         # Encontramos filas de la lista de pesos que tenga de 'from' el nodo 'i' de esta comunidad
#         rows_found = df_edges.loc[df_edges['from'] == i]
#         list_from = rows_found['from'].tolist()
#         list_to = rows_found['to'].tolist()
#         list_weight = rows_found['weight'].tolist()

#         for j in range(len(rows_found)):
#             tuples_from_to.append((list_from[j], list_to[j], list_weight[j]))

#     mdg_iter.add_weighted_edges_from(tuples_from_to)

#     resulting_weights = list(map(lambda x: x[2], tuples_from_to))

#     #Creamos leyenda en función de las relaciones en
#     cNorm = colors.Normalize(vmin = min(resulting_weights), vmax = max(resulting_weights))
#     scalarMap = cmx.ScalarMappable(norm = cNorm, cmap = plt.cm.RdBu)
#     ColorLegend = {'negativo fuerte': -2, 
#                'negativo débil': -1, 
#                'positivo fuerte': 2, 
#                'positivo débil': 1}

#     f = plt.figure(1)
#     ax = f.add_subplot(1,1,1)
#     for label in ColorLegend:
#         ax.plot([0],[0],color=scalarMap.to_rgba(ColorLegend[label]),label=label)

#     plt.legend(loc = 'lower right')
#     nx.draw_shell(mdg_iter,node_size=10,width = 0.1,vmin = min(resulting_weights), vmax = max(resulting_weights), edge_color = resulting_weights ,edge_cmap=plt.cm.RdBu,ax=ax)
#     plt.show()

frozenset({264, 268, 270, 276, 282, 284, 192, 194, 322, 196, 198, 200, 202, 204, 206, 208, 336, 210, 212, 216, 218, 220, 350, 222, 224, 228, 234, 244, 252, 254})


Si calculamos la modularidad de esta partición, con la función que existe en networkx:

In [167]:
print(MDG1A.edges(data = 'weight'))
print(community.quality.modularity(MDG1A,c,weight='weight',resolution=1))

[(207, 191, 1), (207, 194, 2), (207, 201, -1), (207, 202, 1), (207, 205, 2), (207, 206, 2), (207, 284, 2), (207, 311, 2), (206, 191, 2), (206, 194, 1), (206, 196, 1), (206, 197, 2), (206, 198, 2), (206, 199, 2), (206, 201, 1), (206, 202, 2), (206, 204, 1), (206, 207, 1), (206, 209, 1), (206, 210, 1), (206, 211, 1), (206, 212, 1), (206, 215, 1), (206, 218, 1), (206, 220, 1), (206, 223, 1), (206, 257, 2), (206, 264, 2), (206, 284, 2), (206, 285, 1), (206, 311, 2), (205, 193, 1), (205, 194, 2), (205, 195, 2), (205, 199, 1), (205, 201, 1), (205, 202, 2), (205, 204, 2), (205, 207, 2), (204, 191, 1), (204, 192, 1), (204, 193, 1), (204, 194, 1), (204, 195, 2), (204, 196, 2), (204, 197, 2), (204, 198, 1), (204, 199, 2), (204, 202, 1), (204, 203, 1), (204, 205, 1), (204, 206, 1), (204, 207, 1), (204, 209, 2), (204, 215, 1), (204, 218, 2), (204, 220, 1), (204, 270, 2), (203, 191, 1), (203, 192, 1), (203, 193, 1), (203, 194, 1), (203, 195, 2), (203, 196, 1), (203, 197, 1), (203, 198, 1), (203, 19

En cambio, si calculamos la modularidad con nuestra función, que tiene en cuenta los pesos negativos:

In [169]:
print(mymodularity(MDG1A,c))
comunidades_optimizadas = tabu_modularity_optimization(MDG1A,c)
print(mymodularity(MDG1A,comunidades_optimizadas))

0.102863329477903
maximo de vueltas inutiles 1
0.1911845137226421
